In [1]:
!pip install --break-system-packages torch torchvision torchaudio transformers --index-url https://download.pytorch.org/whl/cpu

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cpu


In [ ]:
#function to generate embeddings of the news articles that have affected AAPL stock prices in the time period.




In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# === Load FinBERT Model and Tokenizer ===
model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to("cpu")

# === Correct Label Mapping ===
# FinBERT labels order (index -> label) = {0: positive, 1: negative, 2: neutral}
labels = ["positive", "negative", "neutral"]

# === Define Sentiment Analysis Function ===
def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    probs = torch.nn.functional.softmax(logits, dim=-1).numpy()[0]
    sentiment = labels[probs.argmax()]
    return sentiment, probs

# === Test Sentences ===
texts = [
    "The company reported record profits this quarter.",
    "The market reacted negatively to the CEO's resignation.",
    "The firm's outlook for the next year remains uncertain.",
    "Revenue fell steadily despite global challenges.",
    "Investors are worried about rising interest rates."
]

# === Run Inference ===
for text in texts:
    sentiment, probs = analyze_sentiment(text)
    print(f"📰 {text}")
    print(f"→ Financial Sentiment: {sentiment}")
    print(f"→ Probabilities: {probs.round(3)}\n")


2025-10-17 22:39:11.316042: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-17 22:39:11.316535: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-17 22:39:11.359342: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-17 22:39:12.795788: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

📰 The company reported record profits this quarter.
→ Financial Sentiment: positive
→ Probabilities: [0.949 0.021 0.029]

📰 The market reacted negatively to the CEO's resignation.
→ Financial Sentiment: negative
→ Probabilities: [0.009 0.962 0.029]

📰 The firm's outlook for the next year remains uncertain.
→ Financial Sentiment: neutral
→ Probabilities: [0.029 0.194 0.777]

📰 Revenue fell steadily despite global challenges.
→ Financial Sentiment: negative
→ Probabilities: [0.015 0.971 0.014]

📰 Investors are worried about rising interest rates.
→ Financial Sentiment: negative
→ Probabilities: [0.032 0.887 0.081]

